<a href="https://colab.research.google.com/github/sejhig2/Project_mask_healthy/blob/main/selbstUmsatz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
def sizeCheck(img_path):
    from PIL import Image
    image = Image.open(img_path)
    image_size = image.size
    xL = image_size[0]
    yL = image_size[1]
    print("가로:",xL,"세로:",yL )
    return xL, yL
# img_path = "/content/gdrive/MyDrive/Project_mask/mask_data/Medical Mask/images/0001.jpg"
# sizeCheck(img_path)

In [ ]:
# 한 장에 들어 있는 annotation 개수 확인
def check_annNum(voc_path):
    import json
    with open(voc_path) as json_file:
        voc_info = json.load(json_file)
        ann_list = voc_info["Annotations"]
        annNum = len(ann_list)
    return annNum

voc_path = "/content/gdrive/MyDrive/Project_mask/mask_data/Medical Mask/labels/1805.jpg.json"
print(check_annNum(voc_path))

In [ ]:
# 라밸 이름-> 라밸 순번 추출
def LabelN(LabelName):
    if LabelName == "hijab_niqab" : LabelNum = 0
    elif LabelName == "mask_colorful" : LabelNum = 1
    elif LabelName == "mask_surgical" : LabelNum = 2
    elif LabelName == "face_no_mask" : LabelNum = 3
    elif LabelName == "face_with_mask_incorrect" : LabelNum = 4
    elif LabelName == "face_with_mask" : LabelNum = 5
    elif LabelName == "face_other_covering" : LabelNum = 6
    elif LabelName == "scarf_bandana" : LabelNum = 7
    elif LabelName == "balaclava_ski_mask" : LabelNum = 8
    elif LabelName == "face_shield" : LabelNum = 9
    elif LabelName == "other" : LabelNum = 10
    elif LabelName == "gas_mask" : LabelNum = 11
    elif LabelName == "turban" : LabelNum = 12
    elif LabelName == "helmet" : LabelNum = 13
    elif LabelName == "sunglasses" : LabelNum = 14
    elif LabelName == "eyeglasses" : LabelNum = 15
    elif LabelName == "hair_net" : LabelNum = 16
    elif LabelName == "hat" : LabelNum = 17
    elif LabelName == "goggles" : LabelNum = 18
    elif LabelName == "hood" : LabelNum = 19
    return LabelNum

In [ ]:
# pascal voc 에서 x,y값들과 label을 가져오자
# check_annNum의 출력값을 가져와서 num으로 받을거다.
def xmin_ymaxFromVOC(voc_path,num):
    import json
    with open(voc_path) as json_file:
        voc_info = json.load(json_file)
    print(voc_info["FileName"])
    ann_num = num
    BoundingBox = voc_info["Annotations"][ann_num]
    BoundingBox = BoundingBox["BoundingBox"]
    x_min = BoundingBox[0]
    y_min = BoundingBox[1]
    x_max = BoundingBox[2]
    y_max = BoundingBox[3]
    Label = voc_info["Annotations"][ann_num]
    LabelName = Label["classname"]
    LabelNum = LabelN(LabelName)
    print("x_min:",x_min)
    print("y_min:",y_min)
    print("x_max:",x_max)
    print("y_max:",y_max)
    print("LabelName:",LabelName, " LabelNumber:",LabelNum)
    return x_min, y_min, x_max, y_max, LabelName, LabelNum
voc_path = "/content/gdrive/MyDrive/Project_mask/mask_data/Medical Mask/labels/1805.jpg.json"

empty_list = []
for i in range(check_annNum(voc_path)):
    element = xmin_ymaxFromVOC(voc_path,i)
    empty_list.append(element)
print("annotation list :",empty_list)


In [ ]:
# pascal 형식을 yolo에 맞게 변경
def sizeConvert(xL,yL,x_min, y_min, x_max, y_max):
    x = ((x_max + x_min)/2)/xL
    y = ((y_max + y_min)/2)/yL
    width = (x_max - x_min)/xL
    height = (y_max - y_min)/yL
    return x, y, width, height

- test

In [ ]:
# 폴더 명 바꿔주기


def folder_nameChange(pascal_voc_path):   
    pascal_voc_path

    img_folder_name = "images"
    img_path = pascal_voc_path.replace("labels",img_folder_name,1)
    img_path = img_path.replace(".json","",1)

    new_annotation_folder_name = "annotation_txt"
    txtFile_save_path = pascal_voc_path.replace("labels",new_annotation_folder_name,1)
    txtFile_save_path = txtFile_save_path.replace("json","txt",1)

    return img_path, pascal_voc_path, txtFile_save_path

# 코드 테스트
pascal_voc_path = "/content/gdrive/MyDrive/Project_mask/mask_data/Medical Mask/labels/1805.jpg.json"
folder_nameChange(pascal_voc_path)

In [ ]:
# 위에 함수를 종합해서 파일을 만들어 주는 함수.
def makeFile(img_path,Pascal_voc_path,save_path) : 
    xL, yL = sizeCheck(img_path)

    empty_list = []
    for i in range(check_annNum(Pascal_voc_path)):
        x_min, y_min, x_max, y_max, LabelName, LabelNum = xmin_ymaxFromVOC(Pascal_voc_path,i)
        x, y, width, height = sizeConvert(xL,yL,x_min, y_min, x_max, y_max)
        empty_list.append([LabelNum, x, y, width, height])
    print("annotation list :",empty_list)

    for i in range(len(empty_list)):
        with open(save_path,"a") as f :
            print(empty_list)
            f.write(str(empty_list[i])+"\n")
    return empty_list

# 코드 테스트
voc_path = "/content/gdrive/MyDrive/Project_mask/mask_data/Medical Mask/labels/1805.jpg.json" # json 파일 리스트로 해서 가져올 거다. 지금은 샘플로 하나만.
img_path, pascal_voc_path, txtFile_save_path = folder_nameChange(voc_path)
print(img_path,"\n", pascal_voc_path,"\n", txtFile_save_path)


In [ ]:
from glob import glob 
pascal_voc_path_list = glob("/content/gdrive/MyDrive/Project_mask/mask_data/Medical Mask/labels/*")
print(len(pascal_voc_path_list))

for i in pascal_voc_path_list:
    pascal_voc_path = i
    img_path, pascal_voc_path, txtFile_save_path = folder_nameChange(pascal_voc_path)
    try:
        makeFile(img_path, pascal_voc_path, txtFile_save_path)
    except :
        print("error:",img_path, pascal_voc_path, txtFile_save_path)